In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
DATAPATH = "data/"

# Check the timeseries data and collect the patient ID 

In [3]:
lvl2_train_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_train.pkl"))
lvl2_dev_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_dev.pkl"))
lvl2_test_imputer = pd.read_pickle(os.path.join(DATAPATH,"lvl2_imputer_test.pkl"))
Ys = pd.read_pickle(os.path.join(DATAPATH, "Ys.pkl"))

In [4]:
print("Shape of train, dev, test {}, {}, {}.".format((lvl2_train_imputer.shape), (lvl2_dev_imputer.shape), (lvl2_test_imputer.shape)))
print( 402120 / 24, 57432 / 24, 114936 / 24)

In [5]:
patient_ids = [] # store all patient ids 
for each_entry in Ys.index:
    patient_ids.append(each_entry[0])

In [6]:
print("Number of total patient {}".format(len(patient_ids)))

# Select Clinical Notes

In [7]:
admission_df = pd.read_csv(os.path.join(DATAPATH, "ADMISSIONS.csv"))
noteevents_df = pd.read_csv(os.path.join(DATAPATH, "NOTEEVENTS.csv"))
icustays_df = pd.read_csv(os.path.join(DATAPATH, "ICUSTAYS.csv"))

In [8]:
noteevents_df.groupby(noteevents_df.CATEGORY).agg(['count'])

In [9]:
note_categories = noteevents_df.groupby(noteevents_df.CATEGORY).agg(['count']).index

In [10]:
#selected_note_types = ['Nursing/other', 'Radiology', 'Nursing', 'ECG', 'Physician', 'Echo', 'Respiratory', 'Nutrition']
selected_note_types = []
for each_cat in list(note_categories):
    if each_cat != 'Discharge summary':
        selected_note_types.append(each_cat)

In [12]:
noteevents_df.shape

### Select based on note category

In [13]:
sub_notes = noteevents_df[noteevents_df.CATEGORY.isin(selected_note_types)]

In [14]:
sub_notes.shape

### Drop no chart notes

In [15]:
missing_chardate_index = []
for each_note in sub_notes.itertuples():
    if isinstance(each_note.CHARTTIME, str):
        continue
    if np.isnan(each_note.CHARTTIME):
        missing_chardate_index.append(each_note.Index)
print ("{} of notes does not charttime.".format(len(missing_chardate_index)))
print(sub_notes.shape)

In [16]:
sub_notes.drop(missing_chardate_index, inplace=True )
print(sub_notes.shape)

### Select based on patient id

In [17]:
sub_notes = sub_notes[sub_notes.SUBJECT_ID.isin(patient_ids)] ## select based on patient id

In [18]:
sub_notes.shape

### Select based on time limit (24 hours) 

In [20]:
MIMIC_EXTRACT_DATA = "data/all_hourly_data.h5"
stats = pd.read_hdf(MIMIC_EXTRACT_DATA, 'patients')
TIMELIMIT = 1 # 1day

In [21]:
stats.shape

In [22]:
stats.head()

In [23]:
new_stats = stats.reset_index()

In [24]:
new_stats.rename(columns = {"subject_id": "SUBJECT_ID", "hadm_id": "HADM_ID"}, inplace=True)

In [25]:
print(new_stats.shape, sub_notes.shape)

In [26]:
df_adm_notes = pd.merge(sub_notes[['ROW_ID','SUBJECT_ID','HADM_ID','CHARTTIME', 'CATEGORY', 'TEXT']],
                        new_stats[['SUBJECT_ID','HADM_ID','icustay_id','age','admittime','dischtime', 'deathtime', 
                                  'intime', 'outtime', 'los_icu', 'mort_icu', 'mort_hosp', 'hospital_expire_flag',
                                  'hospstay_seq', 'max_hours']], 
                        on = ['SUBJECT_ID'],
                        how = 'left')

In [27]:
df_adm_notes.head()

In [28]:
df_adm_notes['CHARTTIME'] = pd.to_datetime(df_adm_notes['CHARTTIME'])

In [29]:
df_less_n = df_adm_notes[((df_adm_notes['CHARTTIME']-df_adm_notes['intime']).dt.total_seconds()/(24*60*60))<TIMELIMIT]

In [30]:
df_less_n.shape

# Save clinical notes

In [31]:
pd.to_pickle(df_less_n, "data/sub_notes.p")